# Clustering by GPU

After trying dozens of technologies, we find that GPU acceleration is the most efficient way to speed up clustering currently. 

## Requirements

### CUDA Installation

    Linux

Linux users follow the guide, [NVIDIA CUDA Installation Guide for Linux](https://docs.nvidia.com/cuda/cuda-installation-guide-linux/) to install CUDA.

    Windows

Installation of CUDA on Windows is a bit more complicated, because Stereopy is not supported on Windows now. Following the guide [CUDA on WSL User Guide](https://docs.nvidia.com/cuda/wsl-user-guide/index.html#getting-started-with-cuda-on-wsl), you can run Stereopy with GPU option on WSL.

### RAPIDS on Anaconda

Select the correct version on the homepage of [RAPIDS' official website](https://rapids.ai/start.html). Run following command to build up a specific environment:

In [ ]:
conda create -n rapids-22.12 -c rapidsai -c conda-forge -c nvidia rapids=22.12 python=3.8 cudatoolkit=11.5

<div class="alert alert-info">

**Note**

My real experience installing CUDA on WSL successfully with *NVIDIA Studio Driver WHQL 522.30* according to this [bug](https://forums.developer.nvidia.com/t/cudaruntimeapierror-100-call-to-cudaruntimegetversion-results-in-cuda-error-no-device/234740) reporter’s advice. By the way, this is my personal PC environment with CUDA:
* Intel Core i7-7700k
* NVIDIA-GeForce-RTX-3060（NVIDIA-SMI 522.30; Driver Version: 522.30; CUDA Version: 11.8）
* WSL2 on Windows10(21H2)

</div>

### Stereopy Installation 

Installation through conda command fails in the environment with GPU acceleration, only [PyPI command](https://stereopy.readthedocs.io/en/latest/General/Installation.html#pypi) will succeed.

In [ ]:
pip install stereopy

## Clutsering with GPU

Start with common pipeline:

In [ ]:
import stereo as st
import sys
sys.path.insert(0, '/ldfssz1/ST_BI/USER/lyonlin/stereopy')

# reading data
file_path = '/ldfssz1/ST_BI/USER/stereopy/test/xujunhao/data/SS200000135TL_D1/SS200000135TL_D1.gef'
bin_size = 50
data = st.io.read_gef(file_path, bin_size=bin_size)

# preprocessing
data.tl.cal_qc()
print(data.exp_matrix.shape)
data.tl.normalize_total(target_sum=1e4)
data.tl.log1p()

# get highly variable genes
data.tl.highly_variable_genes(min_mean=0.0125, max_mean=3, min_disp=0.5, res_key='highly_variable_genes', n_top_genes=None)

# PCA
data.tl.pca(use_highly_genes=True, hvg_res_key='highly_variable_genes', n_pcs=20, res_key='pca', svd_solver='arpack')

Cluster demo by CPU:

In [ ]:
data.tl.neighbors(pca_res_key='pca', n_pcs=30, res_key='neighbors', n_jobs=8)
data.tl.umap(pca_res_key='pca', neighbors_res_key='neighbors', res_key='umap', init_pos='spectral')
data.tl.leiden(neighbors_res_key='neighbors', res_key='leiden')

Cluster demo by GPU:

In [ ]:
# note that the parameter method is set to rapids

data.tl.neighbors(pca_res_key='pca', n_pcs=30, res_key='neighbors', n_jobs=8, method='rapids')
data.tl.umap(pca_res_key='pca', neighbors_res_key='neighbors', res_key='umap', init_pos='spectral', method='rapids')
data.tl.leiden(neighbors_res_key='neighbors', res_key='leiden', method='rapids')

## GPU Setting

How to use GPU acceleration on Neighbors, UMAP and Leiden, is shown above, setting the parameter `method` to `rapids`. 

When cluster by Louvain using GPU acceleration, set `flavor` to `rapids`, as below:

In [ ]:
data.tl.neighbors(pca_res_key='pca', n_pcs=30, res_key='neighbors', n_jobs=8, method='rapids')
data.tl.louvain(neighbors_res_key='neighbors', res_key='louvain', flavor='rapids', use_weights=True)